# [MATH2504 Programming of Simulation, Analysis, and Learning Systems at The University of Queensland](https://courses.smp.uq.edu.au/MATH2504/)

## Semester 2, 2021

# Practical B: Julia essentials

In this practical we work on variables, logical statements, conditional statements, loops, generic functions, scope, arrays, input/output, and a few more Julia essentials.

## Simple tasks with arrays

Assume we have a one dimensional array (vector) of arbitrary elements. For example,

```
a = ["hello", "man", "hello", "dog", "hello", "hello", "hello", "fish"]
```

Here the function, `longest_streak()` determines the length of the longest subsequence in the array that has the same element. For example in the case above, `longest_streak(a)` would return `3` because `"hello"` appears for 3 consequative spots and that is more than any other element.

In [ ]:
function longest_streak(a) #a is assumed to be an array
    n = length(a)
    if n == 1
        return 1 #arrays of length 1 have a longest streak of 1
    end
    
    current_value = a[1]
    cs = 1 #current streak
    ls = 1 #longest streak
    for i in 2:n
        if a[i] == current_value
            cs +=1
        else
            current_value = a[i]
            cs = 1
        end
        if cs > ls
            ls = cs
        end
    end
    return ls
end

In [ ]:
#Testing longest_streak() on the simple example above
a = ["hello", "man", "hello", "dog", "hello", "hello", "hello", "fish"]
longest_streak(a)

### Task 1 (Code understanding):
Explain to a peer and/or have them explain to you how the function works. What is the logic in each of the lines.

In [ ]:
#Testing longest_streak() on a slightly longer example

using Random; Random.seed!(2)

a = rand([:x,:y], 20)
@show a
longest_streak(a)

### Task 2 (code manipulaton):
Copy the code of `longest_streak()` to the cell below and carry out the following modifications. After each modification test the code on the (same) pseudo-random sequence of length 20 from above. 

### Task 2a:
In Julia we can take two (or more) lines of the form,
```
x = 3
y = 7
```
and write them as
``
x, y = 3, 7
``
Do this for lines 7, 8, and 9 in the original code which set values for `current_value`, `cs` and `ls`.

### Task 2b:
In Julia it is sometimes common to use [Short-Circuit Evaluation](https://docs.julialang.org/en/v1/manual/control-flow/#Short-Circuit-Evaluation) and transform statements of the form
```
if condition
    do-one-line-action
end
```
to the form
```
(condition) && (do-one-line-action)
```
Do this in two conditional statments of `longest_streak()`

### Task 2c:
It turns out the first test for `length(a) == 1` is redundent because if the length if 1 the `for` loop won't run at all. Argue if this is the case and if so remove the lines associated with this test. Similarly, remove the variable `n` and use `length(a)` direction in the line of the `for` loop. 

In [ ]:
#copy code here


### Task 3 (Timing):

Here is timing of `longest_streak()` on an array of random digits (`1` or `2`) of length $10^7$. Fix the seed at `0` for repeatability.

In [ ]:
using Random; Random.seed!(0) ; a = rand([1,2], 10^7) ; 

@time begin 
    longest_streak(a)
end

Experiment with the array length ranging for $10^6$ to $10^9$ to get a rough feel for the timing.

### Task 4 (Alternative implemintation):
It turns out that using the built in function `accumulate()` and the `mode()` function from the package `StatsBase.jl`, you can have a much shorter function that does the same thing. Here it is called `longest_streak2()`. However it is harder to understand how it operates and it also runs slower.


In [ ]:
using StatsBase
function longest_streak2(a)
    temp = accumulate(+,a[1:end-1] .!= a[2:end])
    sum(temp .== mode(temp))
end

In [ ]:
using Random; Random.seed!(0) ; a = rand([:x,:y,:z], 10^7) ; longest_streak(a)

### Task 4a:
Check it and time it.

### Task 4b:
Explain it.

Neat mathematical problems arise everywhere. 

In [ ]:
Random.seed!(0)
a = rand(["a","b"], 10^7) ; 
@time begin
    longest_streak(a)
end

### Different ways of collecting lists of results

Say we want to study the distributions of the longest streaks when considering arrays of two values where each entry is either `"a"` or `"b"` independently with probability $1/2$. So for array length $n = 2^k$ with $k=1,20$ we repeat the sampling.

In [ ]:
using Random
function compute_results_v1(max_k;N = 10^2)
    Random.seed!(0)
    results = []
    for k in 1:max_k
        n = 2^k
        results_for_k = []
        for _ in 1:N
            a = rand(["a","b"],n)
            result = longest_streak(a)
            push!(results_for_k,result)
        end
        push!(results,results_for_k)
    end
    return results
end;

In [ ]:
results1 = compute_results_v1(20)

In [ ]:
means1 = mean.(results1)

In [ ]:
function compute_results_v2(max_k;N = 10^2)
    Random.seed!(0)
    results = Vector{Vector{Int64}}(undef,max_k)
    for k in 1:max_k
        n = 2^k
        results_for_k = Vector{Int64}(undef,N)
        for i in 1:N
            a = rand(["a","b"],n)
            results_for_k[i] = longest_streak(a)
        end
        results[k] = results_for_k
    end
    return results
end;

In [ ]:
results2 = compute_results_v2(20)

In [ ]:
means2 = mean.(results2)

In [ ]:
function compute_results_v3(max_k;N = 10^2)
    Random.seed!(0)
    results = Matrix{Int64}(undef,max_k,N)
    for k in 1:max_k
        n = 2^k
        for i in 1:N
            a = rand(["a","b"],n)
            results[k,i] = longest_streak(a)
        end
    end
    return results
end;

In [ ]:
results3 = compute_results_v3(20)

In [ ]:
means3 = [mean(results3[k,:]) for k in 1:(size(results3)[1]) ]

In [ ]:
function compute_results_v4(max_k;N = 10^2)
    Random.seed!(0)
    results = Dict{Int,Vector{Int}}()
    for k in 1:max_k
        n = 2^k
        results_for_k = Vector{Int64}(undef,N)
        for i in 1:N
            a = rand(["a","b"],n)
            results_for_k[i] = longest_streak(a)
        end
        results[n] = results_for_k  #notice that the key is n
    end
    return results
end;

In [ ]:
results4 = compute_results_v4(20)

In [ ]:
means4 = [mean(results4[2^k]) for k in 1:length(results4) ]

In [ ]:
function compute_means(max_k;N = 10^2)
    Random.seed!(0)
    means = Dict{Int,Float64}()
    for k in 1:max_k
        n = 2^k
        rm = 0.0 #running mean
        for i in 1:N
            a = rand(["a","b"],n)
            ls = longest_streak(a)
            rm = ((i-1)/i)*rm + 1/i*ls
        end
        means[n] = rm 
    end
    return means
end;

In [ ]:
means_direct = compute_means(20)

In [ ]:
means_direct = sort(means_direct).vals

### Task 6:
Try to determine the output (for a variety of language features).

### Task 6a:

In [ ]:
log2(16), log10(100), log(1/MathConstants.e), cos(π)  #π via \pi + [TAB]

### Task 6b:

In [ ]:
v = 1/√2 + (1/√2)im #√ via \sqrt + [TAB]
v^8

### Task 6c:

In [ ]:
sum([(-1)^n for n in 1:2rand(1:100)])

### Task 6d:

In [ ]:
s = 0
for i in -20:20
    s += i
end
s == 0, s === 0.0

### Task 6e:

In [ ]:
a = 1:5
b = collect(a)
a == b, a === b

### Task 6f:

In [ ]:
x = 5
y = 6
function my_function()
    x = 10
    global y = 12
end

my_function()
x, y

### Task 6g:

In [ ]:
x = 0
while true
    x += 1
    (x == 15) && break
    (x % 5 == 0) && continue
    !(x % 4 == 0) || println("---")
    println(x)
end

### Task 6h:

In [ ]:
a = collect(1:5)
println(a[3:end])
println(a[3:end-1])
println(a[:])

### Task 6i:

In [ ]:
a = [i+j for i in 1:5, j in 1:5]
display(a[2:4,4:end])
display(a[1,:])
display(a[:,end])
println( (min(a...), minimum(a), maximum(a), max(a...)) )

### Task 6j:

In [ ]:
n = 5
function f(x)
    if x == n
        return 0
    else
        return f(x+1) + 1
    end
end

f(0)

### Task 6k:

In [ ]:
function f(x)
    x[2] = "two"
end

a = ["hello",2,[2,3],:my_symbol,2.5]
f(a)
a